In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

import spotipy                          # Set up a config file with ckey and skey. These are available if you go
from spotipy.oauth2 import SpotifyClientCredentials      # to https://developer.spotify.com/, click on Dashboard 
                                        # from the horizontal black menu, login with your normal user info. Click 
# Import Keys                           # on "create an app" if you haven't yet, it doesn't matter what you call it.
from config import ckey, skey            # Then click into
                                        # your project and you should see Client ID and Client Secret. Those are your 
                                        # ckey and skey. 

In [2]:
# Setting up Spotify API info

# Setting up Spotify API info

client_credentials_manager = SpotifyClientCredentials(client_id=ckey, client_secret=skey)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


##TIP: This next section assumes that you have already downloaded the csv files with the Top200 charts for the country you are working on:

Create a subfolder in the folder this notebook is located in called "input_files". Add the owid-covid-data.csv file there, you'll need that later. Then make another subfolder inside input_files called "spotify_top200_charts". Save the csv files you download there.
Go to https://spotifycharts.com
Choose the country you want to work on.
Download Weekly Top200 charts for 2019 and 2020, 1 chart per month. We agreed as a group to download the chart from last week of each month, to keep things consistent. Save them in the "spotify_top200_charts" folder you set up.

In [3]:
# Create dataframe from weekly chart data


path = r"input_files/spotify_top200_charts/*.csv"                # The path requires "".csv" at end of file name. 
                                                                 # This is to prevent the program from blowing up
                                                                 # when it hits some kind of hidden file. 

country_tracks_df = pd.read_csv(glob.glob(path)[0], header=1)    # Sets up **main dataframe** with data from **FIRST** file
string = str(glob.glob(path)[0])                                 # in the folder.
year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)   # This line extracts the year and month from the 
                                                                 # **file name**
country_tracks_df[['Year']] = year_month[0]
country_tracks_df[['Month']] = year_month[1]
country_tracks_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1]) 
Tot_Streams1 = country_tracks_df['Streams'].sum()                # Find out total streams in FIRST file in folder.
country_tracks_df[['Stream %']] = country_tracks_df['Streams'] / Tot_Streams1    # New column with % of streams

for file in glob.glob(path)[1:]:                                 # Now that you have the dataframe set up from the 
        temp_df = pd.read_csv(file, header=1)                    # first file in the folder, this iterates through 
        string = str(file)                                       # remaining files
        year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)
        #print (year_month)
        Tot_Streams2 = temp_df['Streams'].sum()
        temp_df[['Year']] = year_month[0]
        temp_df[['Month']] = year_month[1]
        temp_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1])
        temp_df[['Stream %']] = temp_df['Streams'] / Tot_Streams2
        country_tracks_df = pd.concat([country_tracks_df,        # Adds temperary datafame to end of main dataframe
                                       temp_df])                 # as new rows.  ¿¿¿???? 
        

                                                 
country_tracks_df = country_tracks_df.sort_values(['Year','Month'])  # Sort the new dataframe by year and month




                                                                 # You should get 4,800 rows (24 months x 200 
                                                                 # tracks per month)
country_tracks_df

,Position,Track Name,Artist,Streams,URL,Year,Month,yyyy-mm,Stream %
0,1,Secreto,Anuel AA,2888172,https://open.spotify.com/track/5W83ErFkO3aKAIS...,2019,01,2019-01,0.028116
1,2,Baila Baila Baila,Ozuna,2693711,https://open.spotify.com/track/7uH27oIt4a6cIFC...,2019,01,2019-01,0.026223
2,3,Adan y Eva,Paulo Londra,2448248,https://open.spotify.com/track/6FyRXC8tJUh863J...,2019,01,2019-01,0.023833
3,4,Calma - Remix,Pedro Capó,2216229,https://open.spotify.com/track/5iwz1NiezX7WWjn...,2019,01,2019-01,0.021575
4,5,NI BIEN NI MAL,Bad Bunny,2082242,https://open.spotify.com/track/278kSqsZIiYp8p3...,2019,01,2019-01,0.020270
...,...,...,...,...,...,...,...,...,...
195,196,En Tu Cuerpo - Remix,Lyanno,277680,https://open.spotify.com/track/3dEtfTtCXWW7N8W...,2020,12,2020-12,0.002014
196,197,Amigos,Galvan Real,277638,https://open.spotify.com/track/78OSjd1nAuqqFIH...,2020,12,2020-12,0.002013
197,198,NO TE HAS IDO Y YA TE EXTRAÑO,Aitana,276661,https://open.spotify.com/track/5XJIXRAM4ivaOit...,2020,12,2020-12,0.002006
198,199,Reggaetonera,Anuel AA,272769,https://open.spotify.com/track/6yVkakCN460KOVo...,2020,12,2020-12,0.001978


In [ ]:
# Get Track IDs
track_names = country_tracks_df['Track Name'].to_list()           # Set up list of tracks to iterate through         
track_ids = []                                                    # Empty list to record track IDs into

for track in track_names:                                         # Heads up: with 4800 tracks to process, this takes
    song_results = sp.search(q=track, type='track', limit=1)      # awhile 
    try:
        track_ids.append(song_results['tracks']['items'][0]['id']) # Prevents program from blowing up - few tracks
        print (f"{track} song ID : {song_results['tracks']['items'][0]['id']}")   # Just to let you know it's working
    except IndexError:                                             # lack track ids
        track_ids.append(np.nan)     

Secreto song ID : 5W83ErFkO3aKAIS1WMi6u0
Baila Baila Baila song ID : 7mWFF4gPADjTQjC97CgFVt
Adan y Eva song ID : 6FyRXC8tJUh863JCkyWqtk
Calma - Remix song ID : 5iwz1NiezX7WWjnCgY5TH4
NI BIEN NI MAL song ID : 278kSqsZIiYp8p3QjYAqa8
Amanece song ID : 1LiN0Z98FkR1t0m8KmLcAH
Adictiva song ID : 0z24gh2KYWSfahslybQC4O
MALA song ID : 70uX4beGHc1nQikXveOknp
Mujer Bruja song ID : 4R8TsvDoKSSDVERlaphNBA
Ella Quiere Beber - Remix song ID : 3f5lA8nnc4YYF1qf78WpHw
Pa' Llamar Tu Atención (feat. MC Bin Laden) song ID : 2vEY3cNwXYxIG4fPAoN14q
MIA (feat. Drake) song ID : 116H0KvKr2Zl4RPuVBruDO
7 rings song ID : 6ocbgoVGwYJhOv1GgI9NsF
Imposible song ID : 3ao3OVxHlb3C08vAaApUlm
Ta To Gucci (Remix) song ID : 7kuRvjpeUcn6qNV9IBXC2C
Vas A Quedarte song ID : 0fwIHsKXNEcb57u2um7z9I
Booty song ID : 6OCqV7mvGa48K7UkgkP0p1
Solo de Mi song ID : 5YOkA7qd2983FtBQnP8sYx
Taki Taki (with Selena Gomez, Ozuna & Cardi B) song ID : 4w8niZpiMy6qz1mntFA5uM
Con Calma song ID : 5w9c2J52mkdntKOmRLeM2m
Bubalu song ID : 7dNwJbsT

Perfect song ID : 0tgVpDi06FyKpA1z0VMD4v
Giant (with Rag'n'Bone Man) song ID : 5itOtNx0WxtJmi1TQ3RuRd
Chica Paranormal song ID : 3krzcyMKlRXjVkJFZqMN4t
Y, ¿Si Fuera Ella? - + Es + song ID : 6fiC7Kc7CMN2IKcLDCO6mp
Ojalá song ID : 73Cydqs0F5L35I3Qowtluk
Brindemos song ID : 5P8TXlZCMwru6DaMsIbbO3
No Tengo Nada song ID : 2eiebMTYM93kUajQdIMZOS
PPP - Remix song ID : 3jOn1ZdPldABfKQY4JBhov
Nadie Como Yo song ID : 2JYlYRQDiCMTRHchnSHIyq
La Plata (feat. Lalo Ebratt) song ID : 7Eso5mURNFprb5PmhOXDlf
Azukita song ID : 6tSaMs79oJxK3VZ7QFgoTj
Shotgun song ID : 4ofwffwvvnbSkrMSCKQDaC
Electricity (with Dua Lipa) song ID : 5N4erncE7kuUccm7zEmwzk
Natural song ID : 2FY7b99s15jUprqC0M5NCT
Love Someone song ID : 2JqnpexlO9dmvjUMCaLCLJ
BAGDAD - Cap.7: Liturgia song ID : 1zZ7vl1amOLI4GE5oUn0YB
Moonlight song ID : 0JP9xo3adEtGSdUEISiszL
Princesas song ID : 2geM54lztiqrOuy0wR13Xc
Familia (with Anuel Aa, feat. Bantu) - Spider-Man: Into the Spider-Verse song ID : 4Uy3kNxW2kB8AEoXljEcth
Arms Around You (feat. M

Pa Mí - Remix song ID : 7g8YaUQABMal0zWe7a2ijz
Hipócrita song ID : 4CbANooTL2aTHzJZqQ0Uvk
I Like It song ID : 58q2HKrzhC3ozto2nDdN4z
Solo (feat. Demi Lovato) song ID : 6kPJZM97LwdG9QIsT7khp6
Si Estuviésemos Juntos song ID : 35wvL50xvKpCHEJPxLOLPI
DI MI NOMBRE - Cap.8: Éxtasis song ID : 2Sfl7odHUmG7qc8SbqUyzE
Sin Culpa (feat. DrefQuila) song ID : 03PZ9C7f90TLbDk3dYjBd2
Un Veneno song ID : 4wNxcizGKSTfKm1emmO4yk
Poca Suerte song ID : 5NuVQ1a9vJxMeXJNMpXfZ7
Another One Bites The Dust - Remastered 2011 song ID : 57JVGBtBLCfHw2muk5416J
Duro y Suave song ID : 31M5m42C3EGlbydf9XB5to
In My Mind song ID : 0E9ZjEAyAwOXZ7wJC0PD33
Quiere Beber song ID : 0FDMH2ahuIVzCybwHJK9zh
Callao song ID : 3gu0fRSgFuc4FmrtIv0DnC
Don't Stop Me Now - Remastered 2011 song ID : 7hQJA50XrCWABAu5v6QZ4i
IDGAF song ID : 76cy1WJvNGJTj78UqeA5zr
Fuego song ID : 1eCNttxvk6L9sa3RZzvIUK
Déjalo song ID : 1gDN23O2aqeWX5NqjSnAbV
La player (Bandolera) song ID : 0VKkyBKCXyR99navhVRqcP
Mejor Que Tú song ID : 497YzH5kEsox8IJzN6wOTY

Sola song ID : 0Jr4zQh5i3vrKFEcXrhhxl
Nothing Breaks Like a Heart (feat. Miley Cyrus) song ID : 27rdGxbavYJeBphck5MZAF
Me Niego (feat. Ozuna & Wisin) song ID : 4ggiDYAkJ7Yk8d5xOr7Xjo
Giant (with Rag'n'Bone Man) song ID : 5itOtNx0WxtJmi1TQ3RuRd
bury a friend song ID : 4SSnFejRGlZikf02HLewEF
Está Rico song ID : 2wRkBumdItthjYP9XknImg
Besos En Guerra song ID : 5RPYHXSqcWaaiCbUbyfNAO
High Hopes song ID : 1rqqCSm0Qe4I9rUvWncaom
1, 2, 3 (feat. Jason Derulo & De La Ghetto) song ID : 4QtiVmuA88tPQiCOHZuQ5b
Lobos song ID : 1ZUOlY8R0Ms27uXEmcXT0W
Without Me song ID : 7lQ8MOhq6IN2w8EYcFNSUk
Expertos song ID : 2a2MWHc7gtuecUWU9WIasb
Nadie song ID : 3YdshshEFnDVPZHovHozmE
Love Someone song ID : 2JqnpexlO9dmvjUMCaLCLJ
Me Rehúso song ID : 6De0lHrwBfPfrhorm9q1Xl
TELÉFONO song ID : 0vZi38VepxZWh0ACqYaXio
Te echo de menos song ID : 3gPOFPYLgpvQz0ZMwmv7NE
No te enamores song ID : 2mM3gZ0BbPwPPMelbA8vgt
La Cintura song ID : 6VUJQqyeQhAZHkQbSDqLtI
La Forma en Que Me Miras song ID : 6t94PGXYAuo9GbJEnMpPcW
L

La Plata (feat. Lalo Ebratt) song ID : 7Eso5mURNFprb5PmhOXDlf
Sucker song ID : 22vgEDb5hykfaTwLuskFGD
bury a friend song ID : 4SSnFejRGlZikf02HLewEF
Antes de Morirme (feat. Rosalía) song ID : 4Dl8bEzhAEGbcwkQawS1XG
Que Le De song ID : 08aYFNUTIOMGq93e2VSArQ
Créeme song ID : 0mqayP1Moez8gMZ06oq7rD
Old Town Road (feat. Billy Ray Cyrus) - Remix song ID : 6zpiwiexX8HpFCZLyIWKDv
Un Traguito song ID : 3x45jQfdx5HtXixx4YsoW6
Reggaeton song ID : 6yVkakCN460KOVol9ZazjH
La Romana song ID : 1khmgu0pveJbkbpbkyvcQv
R.I.P. (feat. Rita Ora & Anitta) song ID : 6JqpFboOH2bq9pzaYbkKBn
Pa Olvidarte (feat. Manuel Turizo) - Remix song ID : 2ktoFujjJtantyMoVdBJlX
SOS song ID : 30cSNer6TV8x2utjULVeQ5
Déjala Que Baile (with Alejandro Sanz & Arkano) song ID : 2KK9N6iBAisA0booTu3bGd
El Humo - Canción Original De La Película "Lo Dejo Cuando Quiera" song ID : 22umYdEW0MLHMPPFaZDlxE
Sola song ID : 0Jr4zQh5i3vrKFEcXrhhxl
Caro song ID : 5hTpBe8h35rJ67eAWHQsJx
Bajito song ID : 0Xf523k63exwTTdRmlkiqn
Happier song ID :

...

everything i wanted song ID : 3ZCTVFBt2Brf31RLEnCkWJ
Bésame song ID : 0qlGn8vrV8LkGS0UjMSyLf
Que Tire Pa Lante song ID : 0iFROFTyditGLmb4CtqMBb
Physical song ID : 3AzjcOeAmA57TIOr9zF1ZW
Despacio song ID : 203uubEQA07to1kbro35qQ
Soltera - Remix song ID : 1c0hsvHLELX6y8qymnpLKL
Otro Trago - Remix song ID : 4bTZeO72FwMa6wKOiqoynL
TBT song ID : 4s45VgkknMFkdiGoUcjxQU
Disparos song ID : 2F75mXaavRl0hm3rtM15cq
Me Quedo song ID : 301E9bwin8Gh2PCp0doIDz
Con Calma song ID : 5w9c2J52mkdntKOmRLeM2m
Si Te Vas song ID : 6Y4PDQv4XjYjHLeLmvyOt0
Someone You Loved song ID : 7qEHsqek33rTcFNT9PFqLf
Con La Miel En Los Labios song ID : 2QJcrJNa4KGxQ6bw0MH3Zx
LA CANCIÓN song ID : 0fea68AdmYNygeTGI4RC18
Calma - Remix song ID : 5iwz1NiezX7WWjnCgY5TH4
Lo siento song ID : 2VVqxQhALecXvaArlZSVXr
No Time To Die song ID : 73SpzrcaHk0RQPFP73vqVR
Que Calor (with J Balvin & El Alfa) song ID : 7j7q6tCfEIlTeZtGjuBDT6
PORFA song ID : 5CGRlMZ2SIEkgpnrMqoHnY
No Vuelvas Más song ID : 0mcBgVRVsvESTkEyVHt7Ue
Goteo song ID : 

Rueda song ID : 4NDHYPVJ5zLnR8yYZsMDsu
Hablamos Mañana song ID : 1V8XneTrX6kKeoxYbk4x6h
Ella No Quiere Rosé song ID : 6Aqeqb9rVNA1U6zuuxczQZ
4K song ID : 4xI3U4sDj1TPmO9Iz5TkYp
DOSIS song ID : 0cDumxvNKhydP1Cmon2S6E
A Dónde Vamos song ID : 3iaZkh6Cw1xyUkzJ53eYAV
No Me Conoce - Remix song ID : 4R8BJggjosTswLxtkw8V7P
Soldadito de Hierro song ID : 1wWSi1N5mBif8926JR78Ju
De Tranquilote song ID : 1IDFcb3cjkj2dhv3UaxX9b
Bichiyal song ID : 4j4w4CXm6BSr0s25wAWrrX
Tabú song ID : 471HeGJPKw07MObixON0Um
Presiento song ID : 0gYnWBpArbOOokGLDDhzyG
Roses - Imanbek Remix song ID : 0zLCBJZSiELJf02ucPP9wb
Te Quemaste song ID : 4fj7e6Tc48Bmomwyz2Z2XC
Callaita song ID : 2TH65lNHgvLxCKXM3apjxI
Que Malo song ID : 53v2u51nq3CxiKUNphRzP1
Contando Lunares (feat. Cruz Cafuné) song ID : 6uW0PIRcyZLvkoonm763xD
Pa Los Gustos Los Colores song ID : 6Bd9rs5gv25asdPDRog3ce
Qué Bonito Es Querer song ID : 7Jcvu13qlP4km0uDX9p2pZ
Fantasias song ID : 6mAN61JH0dzyZpWslS11jy
<3 song ID : 6Im9k8u9iIzKMrmV7BWtlF
Memories song

Ponte Pa' Mi song ID : 4C6BlqqMwjFzOXzqpTeoWR
China song ID : 2ksOAxtIxY8yElEWw8RhgK
Si Veo a Tu Mamá song ID : 41wtwzCZkXwpnakmwJ239F
death bed (coffee for your head) song ID : 7eJMfftS33KTjuF7lTsMCx
Atrévete song ID : 1q8NdCAQ9QUjpYiqzdd3mv
TBT song ID : 4s45VgkknMFkdiGoUcjxQU
Me Gusta song ID : 5Xhqh4lwJPtMUTsdBztN1a
Blanco song ID : 2rc7BkzO8qepMFAxHtOrXc
RITMO (Bad Boys For Life) song ID : 6cy3ki60hLwimwIje7tALf
Resistiré song ID : 0XFQb7pNfT90A7fRxjMWgz
4K song ID : 4xI3U4sDj1TPmO9Iz5TkYp
DOSIS song ID : 0cDumxvNKhydP1Cmon2S6E
La Santa song ID : 69vlMrzHwATKzupwNcUPyK
Hola - Remix song ID : 5stPVcRqb4qixbafP9e8lt
Indeciso song ID : 59s0s39NFWScuHDbHytI14
El Efecto - Remix song ID : 74j34STf8L6uADnTN69ohv
Indios y Vaqueros song ID : 3u5GmyJP2MU3tEJgyiAG2T
Physical song ID : 3AzjcOeAmA57TIOr9zF1ZW
Ahora Dice Que Me Ama song ID : 2Z3RqW9jfl62il0R4q8wQy
Toosie Slide song ID : 127QTOFJsJQp5LbJbu3A1y
3 de Abril song ID : 1c2oYPFni5vth1v9M7fYZb
Soldadito de Hierro song ID : 1wWSi1N5mBif

Yo Perreo Sola song ID : 0SqqAgdovOE24BzxIClpjw
CÓMO SE SIENTE - Remix song ID : 2tFwfmceQa1Y6nRPhYbEtC
Una Mala (Remix) song ID : 2maGaB3fNqQnyrG3FDiaW3
Sur y Norte song ID : 2uy5wXDZQcdd89PvDBSfyy
Elegí (feat. Dímelo Flow) song ID : 50ZC4PM7hywH27RcCfViau
Tak Tiki Tak song ID : 7gwwt1IaAWwmB7af0AcuBM
Se iluminaba song ID : 3VPOZWtj8mM89IiRNygxgS
El Efecto song ID : 4SJHHpKhcuXwDsCsPqshXj
Tattoo song ID : 1Srt81FTalOhRw7t7l8Yh8
Rain On Me (with Ariana Grande) song ID : 24ySl2hOPGCDcxBxFIqWBu
Diosa song ID : 3JHpk0DOTOzyh0777JFAky
Enemigos song ID : 3TXNuXIz469yziFDP7LcKD
Atrévete song ID : 1q8NdCAQ9QUjpYiqzdd3mv
Blinding Lights song ID : 0VjIjW4GlUZAMYd2vXMi3b
Confía song ID : 6vYQaurynY7NoHd6Iw85QM
Tu Eres Un Bom Bom - Remix song ID : 1Z9HnKFO3f332B9Ngp0Iyx
Roses - Imanbek Remix song ID : 0zLCBJZSiELJf02ucPP9wb
Morado song ID : 0oMtyJ3w2mCkztYRykM8sg
Si Tú La Quieres song ID : 35qwAQ3oZeKAMhR3hCV9IM
La Difícil song ID : 6NfrH0ANGmgBXyxgV2PeXt
La Cama - Remix song ID : 2tGENA8xef0wABd

5 Sentidos song ID : 7chjOPhoxbpRYSrO1rvdnu
Como Panas song ID : 07MrcfLozSPBfEI8GPOZgc
Falling song ID : 1ZMiCix7XSAbfAJlEZWMCp
La Pared 360 song ID : 2w8SKQCnT2YQK1qCqKl2uW
Te Quemaste song ID : 4fj7e6Tc48Bmomwyz2Z2XC
La Jeepeta - Remix song ID : 1mohfLaTJtB2RplHLQVM70
El Manual song ID : 7kOf0qHO4Rcwk6QJifZDRJ
Hasta Que Dios Diga song ID : 06s3QtMJVXw1AJX3UfvZG1
Caramelo song ID : 3gD4J3RUHy4OUuZ3qAiaiG
PAM song ID : 0eeKDrQxjqNl5bX5VCHDcn
Sur y Norte song ID : 2uy5wXDZQcdd89PvDBSfyy
Carita de Inocente (feat. Myke Towers) - Remix song ID : 5kQre7o8ky5DXeEPcyaCR1
Fabuloso song ID : 3cg2Y9mIiYNdcPTLKaQgK3
Relación song ID : 35UUpTmrcFXNIVIN26ujXl
Mil Tequilas song ID : 3VvyD7SxO6SO3IuCODFAfW
Jangueo song ID : 1bRpSCFv6P2OUhciByeRYR
Pa toda la vida (feat. Mozart La Para) song ID : 6DDb7XRNUzeHmSoTbw5nmm
CÓMO SE SIENTE - Remix song ID : 2tFwfmceQa1Y6nRPhYbEtC
TKN (feat. Travis Scott) song ID : 4w47S36wQGBhGg073q3nt7
PORFA song ID : 5CGRlMZ2SIEkgpnrMqoHnY
Hola, Nena (feat. Omar Montes) s

In [ ]:
# Add Track IDs to dataframe

country_tracks_df['Track ID'] = track_ids                  # Add new column with track IDs

In [ ]:
# Drop empty songs and export dataframe to csv to back it up

clean_country_tracks_df = country_tracks_df.dropna(how='any')           # Use .dropna() to remove rows with missing data

clean_country_tracks_df.to_csv("output_files/1_tracks_with_track_ids.csv", index = False)  # Back up to .csv

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_track_ids_df = pd.read_csv("output_files/1_tracks_with_track_ids.csv")

country_track_ids_df

In [ ]:
# Use API again to get audio features

danceability = []                                     # Set up empty lists to store data in
energy = []
valence = []
tempo = []

for track in country_track_ids_df['Track ID']:           # Heads up: this takes a long time
    
    try:
        feat_results = sp.audio_features([track])
        danceability.append(feat_results[0]['danceability'])
        energy.append(feat_results[0]['energy'])
        valence.append(feat_results[0]['valence'])
        tempo.append(feat_results[0]['tempo'])
        print (f"{track} Valence Score: {feat_results[0]['valence']}")  # Just to let you see it working                                 
    
    except TypeError:                                 # Covers you in case there is missing data
        danceability.append(np.nan)
        energy.append(np.nan)
        valence.append(np.nan)
        tempo.append(np.nan)
        

In [ ]:
# Add audio features to dataframe

country_track_ids_df['Danceability'] = danceability     # Add new columns with audio features
country_track_ids_df['Valence'] = valence
country_track_ids_df['Energy'] = energy
country_track_ids_df['Tempo'] = tempo

# Add new columns with product of % and each feature
country_track_ids_df['Danceability_Stream%'] = danceability * country_track_ids_df['Stream %']    
country_track_ids_df['Valence_Stream%'] = valence * country_track_ids_df['Stream %'] 
country_track_ids_df['Energy_Stream%'] = energy * country_track_ids_df['Stream %'] 
country_track_ids_df['Tempo_Stream%'] = tempo * country_track_ids_df['Stream %'] 

In [ ]:
# Back up dataframe again to .csv
clean_country_track_ids_df = country_track_ids_df.dropna(how='any')          # Use .dropna() to remove rows with missing data

clean_country_track_ids_df.to_csv("output_files/2_tracks_with_audio_features.csv", index=False) #Back up the dataframe to csv again

clean_country_track_ids_df

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_tracks_data_df = pd.read_csv("output_files/2_tracks_with_audio_features.csv")

country_tracks_data_df.head()

In [ ]:
# Use groupby to get average valence of the 200 songs in each month

country_tracks_data_groupby = country_tracks_data_df.groupby(["Year", 'Month'], as_index=False)['Valence_Stream%'].sum()

country_tracks_data_groupby

In [ ]:
# Set up some basic plt formatting configurations

plt.rc('font', size=12)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=20)

In [ ]:
# Plot a comparison of 2019 2020 valence scores

# Set up lists to plot

valence_2019 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2019]

valence_2020 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2020]
valence_2020.drop('Year', inplace=True, axis=1)

fig= plt.figure(figsize=(12,8))                                      # Set up figure size
fig.suptitle('SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH (UK)')         # Set up main title

y_axis = valence_2019['Valence_Stream%']
x_axis = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',                  # Set up x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.plot(x_axis, valence_2019['Valence_Stream%'], label="2019 Weighted Avg Valence", marker='o', color='darkred')  # Plot 2019
plt.plot(x_axis, valence_2020['Valence_Stream%'], label="2020 Weighted Avg Valence", marker='o', color='steelblue')  # Plot 2020

plt.xlabel('Months')                                                 # Set up axis titles
plt.ylabel('Valence Score')

plt.xlim(-0.75, len(x_axis)-0.25)                                    # Set up axis limits
plt.ylim(0, 1)

plt.legend()                                                         # Include the legend

plt.show()

In [ ]:
# Compare valence scores with covide infection rate

covid_df = pd.read_csv("input_files/owid-covid-data.csv")                # read the covid data file

country_covid_df = covid_df.loc[covid_df['location'] == 'United Kingdom']   # Filter for country of your choice

country_covid_df.head()

In [ ]:
# Filter data for 2020, and add a 'month' column

country_covid__2020_df = country_covid_df[country_covid_df.date.str.contains(r'2020.*')]

country_covid__2020_df['Month'] = ''

In [ ]:
# Extract the month from the 'date' column and add it to the new 'month' column, for sorting later

for index, row in country_covid__2020_df.iterrows():
    month, = re.findall(f"2020-(\d\d)-", row['date'])
    country_covid__2020_df.at[index, 'Month'] = int(month)  

country_covid__2020_df.head()

In [ ]:
# Create a groupby to get the sum of new cases in each month

country_covid__2020_groupby = country_covid__2020_df.groupby(['Month'], as_index=False)['new_cases'].sum()
country_covid__2020_groupby


In [ ]:
# TIP: This next section is to add missing months. In the case of New Zealand, there was no data for January
# For other countries, it might vary. Here's how I added January to the dataframe:

# No need to add january in Italy
# country_covid__2020_groupby.loc[-1] = [1, 0]             # This adds a 1 (month of January) in the first columne 
                                                         # index 0), in the last row of the dataframe. 

# country_covid__2020_groupby.index = country_covid__2020_groupby.index + 1   # shifts the index
country_covid__2020_groupby = country_covid__2020_groupby.sort_index()      # sorts by index

country_covid__2020_groupby = country_covid__2020_groupby.rename(columns={"new_cases": "New Cases"})

country_covid__2020_groupby

In [ ]:
# Merge the dataframes into one nice comparison dataframe to scatter plot

country_covid_valence_df = pd.merge(valence_2020, country_covid__2020_groupby, on="Month")

country_covid_valence_df

In [ ]:
# Add a new cases per million column

country_polulation = 67886011  #TIP: This the population of New Zealand. Adjust for your country

country_covid_valence_df['New Cases Per Million'] = country_covid_valence_df['New Cases'] / country_polulation *1000000

country_covid_valence_df

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

fig, ax1 = plt.subplots(figsize=(12,8))                   # Set up subplot figure and size
fig.suptitle('UK: SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH COMPARED TO NEW COVID CASES')

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',       # Set up shared x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax1.set_xlabel('Months')                

ax1.set_ylabel('Average Valence Scores')                  # Set up first plot 
ax1.set_ylim([.2, .8])

ax1_ydata = country_covid_valence_df['Valence_Stream%']
ax1.plot(months, ax1_ydata, label="Weighted Avg Valence Scores", marker='o', color='darkred')                       
                        
ax2 = ax1.twinx()                                         # Set up second plot 
ax2.set_ylabel('New Cases Per Month')
ax2_ydata = country_covid_valence_df['New Cases Per Million']
ax2.set_ylim([0, ax2_ydata.max()+20])
ax2.plot(months, ax2_ydata, label="New Covid Cases Per Million", marker='o', color='steelblue')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()

In [ ]:
# This is a mini function that adds a qualitative label to the correlation r score in the scatter plot

def r_label(r):
    abs_r = abs(r)
    if abs_r >= .8 : return "Very Strong"
    elif abs_r >= .6 : return "Strong"
    elif abs_r >= .4: return "Moderate"
    elif abs_r >= .2: return "Low"
    else: return "Negligible"

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

x_axis = country_covid_valence_df['Valence_Stream%']                                 # Set up axes
y_axis = country_covid_valence_df['New Cases Per Million']

slope, intercept, rvalue, pvalue, stderr = stats.linregress(x_axis, y_axis)  # Get elements of regression equation
regress_values = x_axis * slope + intercept                                  # Calculate regression values

plt.figure(figsize=(12, 8))
plt.title('UK: SPOTIFY LISTENER VALENCE PREFERENCE VS. NEW COVID CASES') # CHANGE TITLE TO REFLECT YOUR COUNTRY

plt.xlabel(f"New Covid Cases")                                               # Set x axis label for subplot                  
plt.ylabel(f"Valence_Stream%")                                                       # Set title for subplot

r = round(stats.pearsonr(x_axis, y_axis)[0],2)                       # Calculate correlation coefficient
rlabel = r_label(r)                                                 # Call function to create a label for the r number

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))    # Ression formula

plt.scatter(x_axis, y_axis, facecolors="darkred",  alpha=.5,        # Plot the scatter chart
label=f"r = {r}\nCorrelation is {rlabel}\n{line_eq}" ) 

plt.plot(x_axis, regress_values, color="steelblue")                 # Plot the regression line    

plt.legend()                                                        # Add the legend
    
plt.savefig("output_files/valence_vs_newcases.png")                 # Save the png file   

plt.show()